# Optimizing a production system

This tutorial will guide you through the optimization functionalities of `prodsys` to optimize the configuration of a production system. With the `prodsys.optimization` package, we can utilize meta-heuristics and mathematical optimization for this task. All algorithms can be conviently used with the `prodsys.models` API. 

For this example, we will use a production system which we will load from a json-file, which can be found in the examples folder of [prodsys' github page](https://github.com/sdm4fzi/prodsys). Download it and store it in the same folder as this notebook. Load the configuration and run a simulation with the following commands:

Let's start at first by loading our production system:

In [2]:
import prodsys
from prodsys.simulation import sim
sim.VERBOSE = 0

production_system = prodsys.adapters.JsonProductionSystemAdapter()	
production_system.read_data('base_configuration.json')

prodsys.adapters.add_default_queues_to_adapter(production_system)

runner = prodsys.runner.Runner(adapter=production_system)
runner.initialize_simulation()
runner.run(2880)
runner.print_results()

  0%|          | 0/2880 [00:00<?, ?it/s]


------------- Throughput -------------

              Output  Throughput
Product_type                    
Product_1        370    9.074785
Product_2        290    7.112669
------------- WIP -------------

Product_type
Product_1    131.548849
Product_2    131.077589
Total        263.968268
Name: WIP, dtype: float64

------------- Throughput time -------------

Product_type
Product_1    549.864540
Product_2    671.308317
Name: Throughput_time, dtype: float64

------------- Resource states -------------

                    time_increment  resource_time  percentage
Resource Time_type                                           
M1       PR            1012.527092    2879.447441   35.163937
         SB            1866.920349    2879.447441   64.836063
M2       PR            2847.310178    2879.447441   98.883909
         SB               7.137263    2879.447441    0.247869
         UD              25.000000    2879.447441    0.868222
M3       PR            2850.035419    2879.447441   98.978

As already concluded in the seccond tutorial, production system configurations can be suboptimal for a certain load of products. In this example, we also see that resoures M2, M3, M4 are very heavily utilized, whereas resource M1 has only a productivy of 33.4%. In order to satify the product needs of our customers and to balance the load on the resources more evenly, we want to find a more suitable configuration with the `prodsys.optimization` package. However, for starting optimization, we also need to provide an optimization scenario, that models constraints, options, information and the objectives. Let's start by creating the constraints of the scenario with the `prodsys.models` API:

In [3]:
from prodsys.models import scenario_data

constraints = scenario_data.ScenarioConstrainsData(
    max_reconfiguration_cost=100000,
    max_num_machines=8,
    max_num_processes_per_machine=3,
    max_num_transport_resources=2
)

As you can see, the constraints consist of the maximum cost for reconfigruation and the maximumm number of machines, processes per machine and transport resources. Next, we define the options of our scenario for optimization:

In [4]:
positions = [[x*4, y*4] for x in range(4) for y in range(10)]
options = scenario_data.ScenarioOptionsData(
    transformations=[scenario_data.ReconfigurationEnum.PRODUCTION_CAPACITY],
    machine_controllers=["FIFO", "SPT"],
    transport_controllers=["FIFO", "SPT_transport"],
    routing_heuristics=["shortest_queue"],
    positions=positions
)

We specify in the scenario options the transformations that can be performed by the optmizer, which control policies and routing heuristics are available and what kind of positions are available to place resources. By choosing the transformation `PRODUCTION_CAPACITY`, the optimizer can add, remove or move production resources from the system or processes from single production resources.

At last, we need to specify our info for optimization:

In [5]:
info = scenario_data.ScenarioInfoData(
    machine_cost=40000,
    transport_resource_cost=20000,
    process_module_cost=4000,
    time_range=24*60
)

The scenario info contains information about the cost for machines, transport resources and process modules. Additionally, we specify a time range. This value is the time used for evalutation of created configurations during simulation. Since many evaluations are performed during optimization, this parameter can significantly influence the optimmization Time. For now, we specified it to one day. Lastly we can define the objectives used for optimization:

In [6]:
from prodsys.models.performance_indicators import KPIEnum
objectives = [scenario_data.Objective(
    name=KPIEnum.THROUGHPUT
)]

Currently, only reconfiguration cost, throughput time, WIP and throughput can be optimized. Yet, similar logic can also be used for optimizing the productivity. With all this data defined, we can now create our optimization scenario and additionally add it to our production system:

In [7]:
scenario = scenario_data.ScenarioData(
    constraints=constraints,
    options=options,
    info=info,
    objectives=objectives
)
production_system.scenario_data = scenario

Next, we define the hyper parameters for our optimization. At first, we will use evolutionary algorithm for our optimization, because it allows parallelization. The hyper parameters for optimization are strongly problem dependant and need to be adjusted accordingly. For this example, we will use the following parameters and run the optimization for 10 generations. Note, that this can take some time...

In [8]:
from prodsys.optimization import evolutionary_algorithm_optimization
from prodsys.optimization import evolutionary_algorithm
from prodsys.simulation import sim
sim.VERBOSE = 0

hyper_parameters = evolutionary_algorithm.EvolutionaryAlgorithmHyperparameters(
    seed=0,
    number_of_generations=10,
    population_size=16,
    mutation_rate=0.2,
    crossover_rate=0.1,
    number_of_processes=4
)
evolutionary_algorithm_optimization(
    production_system,
    hyper_parameters,
)

Best Performance:  498.0
Average Performance:  315.75
Generation 1 ________________
Best Performance:  498.0
Average Performance:  401.625
Generation 2 ________________
Best Performance:  498.0
Average Performance:  -5819.0
Generation 3 ________________
Best Performance:  503.0
Average Performance:  473.9375
Generation 4 ________________
Best Performance:  503.0
Average Performance:  -5782.375
Generation 5 ________________
Best Performance:  503.0
Average Performance:  -12083.625
Generation 6 ________________
Best Performance:  503.0
Average Performance:  -5782.375
Generation 7 ________________
Best Performance:  503.0
Average Performance:  -12081.3125
Generation 8 ________________
Best Performance:  503.0
Average Performance:  499.625
Generation 9 ________________
Best Performance:  503.0
Average Performance:  499.625
Generation 10 ________________
Best Performance:  503.0
Average Performance:  -24622.75


All algorithms in `prodsys` can be utilized with the same interface. Also available are the following algorithms:
- `prodsys.optimization.simulated_annealing`: simulated annealing optimization for all transformations
- `prodsys.optimization.tabu_search`: tabu search for all transformations
- `prodsys.optimization.math_opt`: mathematical optimization with Gurobi, allows only optimization of the production capacity

We see in the output, that the algorithm is running and that new best solutions with a higher performance are found. We can analyze them now and see, if we can find a better configuration for our production system. Optimization core results of the objective for the individual solutions and the solutions themselves are saved as default in a `results` folder to make sure that interruptions in optimization won't delete all results. We can load them with the following command:

In [10]:
from prodsys.optimization import optimization_analysis

df = optimization_analysis.read_optimization_results_file_to_df("results/optimization_results.json", "evolutionary")
df.head()

,Generation,population_number,ID,agg_fitness,time,KPI_0,optimizer
1,0,1,c239c703-0014-11ee-ad97-010101010000,382.0,36.275489,382.0,evolutionary
2,0,2,c23ae71f-0014-11ee-9880-010101010000,246.0,36.280622,246.0,evolutionary
3,0,3,c23baece-0014-11ee-b5bf-010101010000,247.0,36.285604,247.0,evolutionary
4,0,4,c23c4e63-0014-11ee-a54c-010101010000,23.0,36.293567,23.0,evolutionary
5,0,5,c23da86d-0014-11ee-8baf-010101010000,338.0,36.299914,338.0,evolutionary


`prodsys` allows us to load the optimization results as a data frame and analyze them. In this case, we just want to see the best solution found by the algorithm:

In [11]:
df.sort_values(by=["agg_fitness"], ascending=False).head()

,Generation,population_number,ID,agg_fitness,time,KPI_0,optimizer
47,3,7,d5ad3205-0014-11ee-9880-010101010000,503.0,68.906486,503.0,evolutionary
59,5,4,d5ad3205-0014-11ee-9880-010101010000,503.0,80.29236,503.0,evolutionary
74,7,4,d5ad3205-0014-11ee-9880-010101010000,503.0,100.402081,503.0,evolutionary
83,8,3,d5ad3205-0014-11ee-9880-010101010000,503.0,111.538516,503.0,evolutionary
86,9,1,d5ad3205-0014-11ee-9880-010101010000,503.0,116.622233,503.0,evolutionary


In [12]:
import os 

# Find all files i the result folder that contain the ID of the best individual
best_individual_ID = df.sort_values(by=["agg_fitness"], ascending=False).head()["ID"].values[0]
best_individual_ID = str(best_individual_ID)
files = os.listdir("results")
files = [file for file in files if best_individual_ID in file]
new_production_system = prodsys.adapters.JsonProductionSystemAdapter()
new_production_system.read_data("results/" + files[0])

In [13]:
runner = prodsys.runner.Runner(adapter=new_production_system)
runner.initialize_simulation()
runner.run(2880)

runner.print_results()


------------- Throughput -------------

              Output  Throughput
Product_type                    
Product_1        587   14.392635
Product_2        443   10.861903
------------- WIP -------------

Product_type
Product_1    43.855144
Product_2    25.727386
Total        68.876164
Name: WIP, dtype: float64

------------- Throughput time -------------

Product_type
Product_1    163.415071
Product_2    135.268264
Name: Throughput_time, dtype: float64

------------- Resource states -------------

                                                time_increment  resource_time   
Resource                             Time_type                                  
TR1                                  PR            2022.705778    2879.983667  \
                                     SB             847.277890    2879.983667   
                                     UD              10.000000    2879.983667   
ace20bd7-0014-11ee-a439-010101010000 PR            2750.093916    2879.983667   
         

When comparing the results from the original production system and the new one, we see that a machine was added. However, the machines are still heavily utilized. Most likely, the optimizer did just not find a good solution, because we only ran it for 10 generations and for a small population size. Increasing these will take longer, but will more likely find better solutions. 